In [1]:
%matplotlib inline
import numpy as np
import time
import h5py
import keras
import pandas as pd
import math
import joblib
import matplotlib.pyplot as plt

from fuel.datasets.hdf5 import H5PYDataset

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from IPython.display import display

from keras.layers import Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Activation, Dropout
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback


Using TensorFlow backend.


In [2]:
ftrain = H5PYDataset("../../data/cifar10/cifar10.hdf5", which_sets=('train',))
X_train, y_train = ftrain.get_data(ftrain.open(), slice(0, ftrain.num_examples))
X_train = np.moveaxis(X_train[:], 1, 3) / 255.

ftest = H5PYDataset("../../data/cifar10/cifar10.hdf5", which_sets=('test',))
X_test, y_test = ftest.get_data(ftest.open(), slice(0, ftest.num_examples))
X_test = np.moveaxis(X_test[:], 1, 3) / 255.
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [3]:
# input image dimensions
img_rows, img_cols, img_chns = 32, 32, 3

# number of convolutional filters to use
num_classes = 10

batch_size = 100
original_img_size = (img_rows, img_cols, img_chns)
epochs = 500

In [4]:
def create_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    opt = RMSprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

create_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

In [5]:
results = []
for sample_size in [1000, 2000, 5000, 10000, 25000, 50000]:
    start = time.time()
    print('Fitting with sample_size: {}'.format(sample_size))
   
    if sample_size < len(X_train):
        sss = StratifiedShuffleSplit(n_splits=2, test_size=sample_size / len(X_train), random_state=0)
        _, index = sss.split(X_train, y_train)
        X, y = X_train[index[1]], y_train[index[1]]
    else:
        X, y = X_train, y_train
   
    y = np_utils.to_categorical(y)
    model = create_model()
    model.fit(X, y, shuffle=True, 
              epochs=epochs,
              batch_size=batch_size,
              verbose=0,
              callbacks=[TQDMNotebookCallback(), 
                         EarlyStopping(monitor='loss', min_delta=0.01, patience=50)])
    
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    score = accuracy_score(y_test, y_pred)
    
    end = time.time()
    elapsed = end - start
    print(' * Accuracy: %.1f %%' % (100. * score))
    print(' * Fit time elapsed: %.1fs' % elapsed)
    results.append({'sample_size': sample_size, 'accuracy': score, 'time': elapsed})

Fitting with sample_size: 1000

 * Accuracy: 43.4 %
 * Fit time elapsed: 64.2s
Fitting with sample_size: 2000

 * Accuracy: 48.4 %
 * Fit time elapsed: 98.7s
Fitting with sample_size: 5000

 * Accuracy: 61.0 %
 * Fit time elapsed: 231.8s
Fitting with sample_size: 10000

 * Accuracy: 67.3 %
 * Fit time elapsed: 346.6s
Fitting with sample_size: 25000

 * Accuracy: 76.8 %
 * Fit time elapsed: 601.1s
Fitting with sample_size: 50000

 * Accuracy: 80.7 %
 * Fit time elapsed: 921.4s


In [6]:
df = pd.DataFrame(results)
display(df)
df.to_csv('cnn_results.csv', index=False)

,accuracy,sample_size,time
0,0.4335,1000,64.155168
1,0.4844,2000,98.667964
2,0.6097,5000,231.792324
3,0.6731,10000,346.644258
4,0.7675,25000,601.067277
5,0.8073,50000,921.351678
